In [18]:
import torch
from Trainer.head_trainer import Trainer
from utils.dataloader import CreateDataset
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score,f1_score, recall_score, precision_score, confusion_matrix
import numpy as np
from architecture.simplebert import BertLinear1HEAD

In [4]:
test_set = pd.read_table('dataset/Test/localch.tsv')
test_set.drop(columns=['Unnamed: 2',"Unnamed: 3","Unnamed: 4"], inplace=True)
test_set.columns = ['text','label_x']
batch_size = 128
bert_name="FacebookAI/xlm-roberta-base"
test_data_loader  = CreateDataset(test_set['text'], test_set['label_x'], bert_name, batch_size=batch_size).todataloader()

In [32]:
test_set = pd.read_table('dataset/Test/Table-S6-career-2019.tsv')
test_set = test_set.iloc[:, :2]
test_set.columns = ['text','label_x']

In [23]:
class Predict:
    def __init__(self,name, test_data_loader,model_path):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.bertcnn=BertLinear1HEAD(name).to(self.device)
        self.bertcnn.load_state_dict(torch.load(model_path))  # Load the state dictionary
        self.bertcnn.eval()


        self.test_data_loader  = test_data_loader
        
        
    
    def predictions_labels(self,preds,labels):
        pred = np.argmax(preds,axis=1).flatten()
        label = labels.flatten()
        return pred,label
    
    def eval(self):
        all_true_sent = []
        all_pred_sent = []

        with torch.no_grad():
            for batch in tqdm(self.test_data_loader):
                b_input_ids = batch[0].to(self.device)
                b_input_mask = batch[1].to(self.device)
                b_sent = batch[2].to(self.device)
                #b_clas = batch[3].to(self.device)

                self.sent_predictions = self.bertcnn(b_input_ids,b_input_mask)
                #loss1 = self.criterion(sent_predictions, b_sent) 
                #loss2 = self.criterion(clas_predictions, b_clas) 

                #t_loss = loss1

                self.sent_predictions = self.sent_predictions.detach().cpu().numpy()
                #clas_predictions = clas_predictions.detach().cpu().numpy()

                label_sent = b_sent.to('cpu').numpy()
                #label_clas = b_clas.to('cpu').numpy()

                pred1, true1 = self.predictions_labels(self.sent_predictions,label_sent)
                #pred2, true2 = self.predictions_labels(clas_predictions,label_clas)

                all_pred_sent.extend(pred1)
                #all_pred_clas.extend(pred2)

                all_true_sent.extend(true1)
                #all_true_clas.extend(true2)

            val_accuracy_sent = accuracy_score(all_pred_sent,all_true_sent)
            #val_accuracy_clas = accuracy_score(all_pred_clas,all_true_clas)

            sent_f1_score = f1_score(all_pred_sent,all_true_sent,average='macro')
            sent_f1_scorew = f1_score(all_pred_sent,all_true_sent,average='weighted')
            #clas_f1_score = f1_score(all_pred_clas,all_true_clas,average='macro')
            #clas_f1_scorew = f1_score(all_pred_clas,all_true_clas,average='weighted')
            precision_score_sent = precision_score(all_pred_sent,all_true_sent) 
            recall_score_sent = recall_score(all_pred_sent,all_true_sent)
            accs = (val_accuracy_sent)
            f1s= (sent_f1_score,sent_f1_scorew)
        return  accs,  f1s, precision_score_sent, recall_score_sent



In [25]:
predicting = Predict(bert_name, test_data_loader, "model/Epoch-5-2-head-linear-smart-10_0-redo.pt")
res = predicting.eval()

  0%|          | 4/1027 [00:20<1:29:14,  5.23s/it]


KeyboardInterrupt: 

In [42]:
df = test_set[test_set['label_x'] == 1]
df.to_excel('dataset/Test/table-s6-career-2019-1.xlsx')